In [19]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!unzip /content/drive/MyDrive/Dataset.zip -d /content/drive/MyDrive/Dataset

Archive:  /content/drive/MyDrive/Dataset.zip
  inflating: /content/drive/MyDrive/Dataset/README.dataset.txt  
  inflating: /content/drive/MyDrive/Dataset/README.roboflow.txt  
   creating: /content/drive/MyDrive/Dataset/train/
  inflating: /content/drive/MyDrive/Dataset/train/_annotations.coco.json  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_10_png.rf.e93923fab7a4d852bcc87747de90abe8.jpg  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_11_png.rf.2322955a1c5bf5b13f6306e454aea228.jpg  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_12_png.rf.9f915fdb7868a316601718ff0adff6a3.jpg  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_13_png.rf.971647d529006f7913ce77aced0c48e3.jpg  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_14_png.rf.cd7221022ed1be4b57e6279da896e9c4.jpg  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_16_png.rf.1dc07b6d124966ae7b70b364cde62b78.jpg  
 extracting: /content/drive/MyDrive/Dataset/train/cut1_17_png.rf.0ef4a2dae

In [18]:
# Install pycocotools
!pip install pycocotools

import torch
import torchvision
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.transforms import Compose, ToTensor, RandomHorizontalFlip, RandomRotation, ColorJitter
from torchvision.datasets import CocoDetection
from torch.utils.data import DataLoader, random_split
from torch.optim import SGD
from torch.optim.lr_scheduler import StepLR
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
import random
import os
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

# Set seed
random.seed(42)
torch.manual_seed(42)

# Collate function
def collate_fn(batch):
    images, targets = zip(*batch)
    # Ensure targets are lists of dictionaries
    targets = [t if isinstance(t, list) else [t] for t in targets]
    # Convert COCO format to Faster R-CNN format
    formatted_targets = []
    for t in targets:
        formatted_t = {}
        for ann in t:  # Process each annotation (should be one per image)
            formatted_t['boxes'] = torch.tensor([[ann['bbox'][0], ann['bbox'][1],
                                                 ann['bbox'][0] + ann['bbox'][2],
                                                 ann['bbox'][1] + ann['bbox'][3]]],
                                                dtype=torch.float32)
            formatted_t['labels'] = torch.tensor([ann['category_id']], dtype=torch.int64)
            formatted_t['image_id'] = torch.tensor([ann['image_id']], dtype=torch.int64)
        formatted_targets.append(formatted_t)
    return images, formatted_targets

# Paths (update these)
images_root = '/content/drive/MyDrive/Dataset/train'  # E.g., '/content/drive/MyDrive/images/'
coco_ann_file = '/content/drive/MyDrive/Dataset/train/_annotations.coco.json'  # E.g., '/content/drive/MyDrive/annotations.json'

# Verify dataset
if not os.path.exists(coco_ann_file) or not os.path.exists(images_root):
    raise FileNotFoundError("Check paths: images_root or coco_ann_file not found")
image_files = os.listdir(images_root)
if len(image_files) != 95:
    print(f"Warning: Found {len(image_files)} images, expected 95")
coco = COCO(coco_ann_file)
json_files = set(coco.imgs[i]['file_name'] for i in coco.imgs)
extracted_files = set(os.listdir(images_root))
missing = json_files - extracted_files
if missing:
    print(f"Missing images: {missing}")
else:
    print("All images found")

# Debug dataset
full_dataset = CocoDetection(root=images_root, annFile=coco_ann_file, transform=ToTensor())
print("Debugging dataset:")
for i, (img, target) in enumerate(full_dataset):
    print(f"Sample {i}: Target = {target}")
    if i == 2:
        break

# Balanced split
def balanced_split(dataset, train_size, test_size):
    cut_indices = [i for i, (_, target) in enumerate(dataset) if target[0]['category_id'] == 1]
    flash_indices = [i for i, (_, target) in enumerate(dataset) if target[0]['category_id'] == 2]
    if len(cut_indices) < 5 or len(flash_indices) < 5:
        raise ValueError("Not enough images for balanced test split")
    random.shuffle(cut_indices)
    random.shuffle(flash_indices)
    test_indices = cut_indices[:5] + flash_indices[:5]
    train_indices = list(set(range(len(dataset))) - set(test_indices))
    return random_split(dataset, [len(train_indices), len(test_indices)], generator=torch.Generator().manual_seed(42))

# Transforms
train_transform = Compose([
    ToTensor(),
    RandomHorizontalFlip(0.5),
    RandomRotation(10),
    ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)
])
test_transform = ToTensor()

# Load dataset
full_dataset = CocoDetection(root=images_root, annFile=coco_ann_file, transform=test_transform)
train_dataset, test_dataset = balanced_split(full_dataset, 85, 10)
train_dataset.dataset.transform = train_transform

# DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=2, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=2, collate_fn=collate_fn)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Model
model = fasterrcnn_resnet50_fpn(pretrained=True)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes=3)
model.to(device)

# Optimizer and scheduler
params = [p for p in model.parameters() if p.requires_grad]
optimizer = SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
scheduler = StepLR(optimizer, step_size=3, gamma=0.1)

# Training
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, targets in train_loader:
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) if torch.is_tensor(v) else v for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        running_loss += losses.item()
    scheduler.step()
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

# Save model
torch.save(model.state_dict(), 'defect_detection_model.pth')
print("Model saved to 'defect_detection_model.pth'")

# Inference and visualization
model.eval()
class_names = ['background', 'Cut', 'Flash']
os.makedirs('test_predictions', exist_ok=True)

coco_gt = COCO(coco_ann_file)
coco_results = []

with torch.no_grad():
    for idx, (images, targets) in enumerate(test_loader):
        images = [img.to(device) for img in images]
        preds = model(images)

        img = images[0].cpu().permute(1, 2, 0).numpy()
        if img.max() <= 1.0:
            img = (img * 255).astype(np.uint8)
        fig, ax = plt.subplots(1)
        ax.imshow(img)

        img_id = targets[0]['image_id'].item()
        file_name = test_dataset.dataset.coco.imgs[img_id]['file_name']
        print(f"\nTest Image {idx+1} (File: {file_name}):")
        gt_box = [targets[0]['boxes'][0][0], targets[0]['boxes'][0][1],
                  targets[0]['boxes'][0][2] - targets[0]['boxes'][0][0],
                  targets[0]['boxes'][0][3] - targets[0]['boxes'][0][1]]  # Convert to [x, y, w, h]
        print(f"Ground Truth: {class_names[targets[0]['labels'][0]]}, Box: {gt_box}")

        for pred in preds:
            boxes = pred['boxes'].cpu().numpy()
            labels = pred['labels'].cpu().numpy()
            scores = pred['scores'].cpu().numpy()

            for box, label, score in zip(boxes, labels, scores):
                if score > 0.5:
                    print(f"  - Predicted: Class: {class_names[label]}, Score: {score:.2f}, Box: {box}")
                    rect = patches.Rectangle((box[0], box[1]), box[2]-box[0], box[3]-box[1], linewidth=2, edgecolor='r', facecolor='none')
                    ax.add_patch(rect)
                    ax.text(box[0], box[1], f"{class_names[label]} {score:.2f}", bbox=dict(facecolor='white', alpha=0.5))

            for box, label, score in zip(boxes, labels, scores):
                if score > 0.5:
                    coco_results.append({
                        "image_id": img_id,
                        "category_id": label,
                        "bbox": [float(box[0]), float(box[1]), float(box[2]-box[0]), float(box[3]-box[1])],
                        "score": float(score)
                    })

        plt.axis('off')
        plt.savefig(f'test_predictions/test_image_{idx+1}_{file_name}')
        plt.close()
        print(f"Visualization saved to 'test_predictions/test_image_{idx+1}_{file_name}'")

# Compute mAP
if coco_results:
    coco_dt = coco_gt.loadRes(coco_results)
    coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
    coco_eval.params.iouThrs = np.array([0.5])
    coco_eval.params.imgIds = [test_dataset.dataset.coco.imgs[i]['id'] for i in test_dataset.indices]
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()
    print(f"Test mAP@0.5: {coco_eval.stats[0]:.4f}")
else:
    print("No predictions above threshold for mAP calculation.")

# Download outputs
from google.colab import files
files.download('defect_detection_model.pth')
!zip -r test_predictions.zip test_predictions/
files.download('test_predictions.zip')

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
All images found
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Debugging dataset:
Sample 0: Target = [{'id': 0, 'image_id': 0, 'category_id': 1, 'bbox': [561, 267, 37, 54.5], 'area': 2016.5, 'segmentation': [], 'iscrowd': 0}]
Sample 1: Target = [{'id': 1, 'image_id': 1, 'category_id': 1, 'bbox': [452, 187, 49, 48], 'area': 2352, 'segmentation': [], 'iscrowd': 0}]
Sample 2: Target = [{'id': 2, 'image_id': 2, 'category_id': 1, 'bbox': [375, 363, 41, 59.5], 'area': 2439.5, 'segmentation': [], 'iscrowd': 0}]
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Using device: cuda
Epoch [1/10], Loss: 0.3661
Epoch [2/10], Loss: 0.2647
Epoch [3/10], Loss: 0.2648
Epoch [4/10], Loss: 0.2622
Epoch [5/10], Loss: 0.2561
Epoch [6/10], Loss: 0.2461
Epoch [7/10], Loss: 0.2404
Epoch [8/10], Loss: 0.2361
Epoch [9/10], Loss: 0.2359
Epoch [10/10], Loss: 0.23

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

  adding: test_predictions/ (stored 0%)
  adding: test_predictions/test_image_8_flash2_5_png.rf.710edfcdbdd2d6d6ea9bdc3cf3e05553.jpg (deflated 40%)
  adding: test_predictions/test_image_1_cut1_3_png.rf.303085f18402fedbe55629905e37cdc4.jpg (deflated 40%)
  adding: test_predictions/test_image_5_flash1_8_png.rf.1492fa6599ce249487594a6b2e7207d1.jpg (deflated 39%)
  adding: test_predictions/test_image_9_flash1_1_png.rf.023a77ad22904bdb26d59a117891fe0a.jpg (deflated 40%)
  adding: test_predictions/test_image_4_cut1_6_png.rf.6ffd8dd7bdc901e69026880b32ad1446.jpg (deflated 42%)
  adding: test_predictions/test_image_10_flash1_11_png.rf.dc5b26f964ae8cd1170b20d2b212cde5.jpg (deflated 37%)
  adding: test_predictions/test_image_2_flash2_18_png.rf.4a3b8afe0cb430dc3aa0de25aedca8e7.jpg (deflated 40%)
  adding: test_predictions/test_image_7_cut2_13_png.rf.f42995a226dd05307742f952d4547cbb.jpg (deflated 41%)
  adding: test_predictions/test_image_3_flash2_11_png.rf.7c2a5f23237db59113bcf540aac6ba98.jpg (def

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
from google.colab import files
files.download('defect_detection_model.pth')
!zip -r test_predictions.zip test_predictions/
files.download('test_predictions.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

updating: test_predictions/ (stored 0%)
updating: test_predictions/test_image_8_flash2_5_png.rf.710edfcdbdd2d6d6ea9bdc3cf3e05553.jpg (deflated 40%)
updating: test_predictions/test_image_1_cut1_3_png.rf.303085f18402fedbe55629905e37cdc4.jpg (deflated 40%)
updating: test_predictions/test_image_5_flash1_8_png.rf.1492fa6599ce249487594a6b2e7207d1.jpg (deflated 39%)
updating: test_predictions/test_image_9_flash1_1_png.rf.023a77ad22904bdb26d59a117891fe0a.jpg (deflated 40%)
updating: test_predictions/test_image_4_cut1_6_png.rf.6ffd8dd7bdc901e69026880b32ad1446.jpg (deflated 42%)
updating: test_predictions/test_image_10_flash1_11_png.rf.dc5b26f964ae8cd1170b20d2b212cde5.jpg (deflated 37%)
updating: test_predictions/test_image_2_flash2_18_png.rf.4a3b8afe0cb430dc3aa0de25aedca8e7.jpg (deflated 40%)
updating: test_predictions/test_image_7_cut2_13_png.rf.f42995a226dd05307742f952d4547cbb.jpg (deflated 41%)
updating: test_predictions/test_image_3_flash2_11_png.rf.7c2a5f23237db59113bcf540aac6ba98.jpg (def

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>